## First level GLM analysis

This script performs subject level modeling of BOLD response. Script features: 
- reads BIDS dataset, loads imaging files, confounds and parametric modulations
- for each subject and session:
    - creates parametrically modulated regressors
    - creates full first level GLM
    - estimate and save statistical maps for predefined contrasts

---
**Last update**: 12.02.2020 

In [ ]:
%matplotlib inline
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
from collections import namedtuple

import nibabel as nib
from bids import BIDSLayout
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, show
from nistats.first_level_model import FirstLevelModel
from nistats.reporting import plot_design_matrix, make_glm_report
from nistats.thresholding import map_threshold
from nistats.design_matrix import make_first_level_design_matrix

path_root = os.environ.get('DECIDENET_PATH')
path_code = os.path.join(path_root, 'code')
if path_code not in sys.path:
    sys.path.append(path_code)
from dn_utils.behavioral_models import load_behavioral_data                    
from dn_utils.glm_utils import Regressor, my_make_first_level_design_matrix
from dn_utils.plotting import plot_correlation_between_regressors

In [ ]:
# Directory to save first-level output
path_out = os.path.join(path_root, 
                        'data/main_fmri_study/derivatives/nistats/first_level_output')
os.makedirs(path_out, exist_ok=True)

# Load behavioral data
path_beh = os.path.join(path_root, 'data/main_fmri_study/sourcedata/behavioral')
beh, meta = load_behavioral_data(path=path_beh)
n_subjects, n_conditions, n_trials, _ = beh.shape

### Query neuroimaging dataset

Using BIDSLayout object query BIDS dataset to pull out necessary files.
- `anat_files`: sorted list of preprocessed T1w images
- `fmri_files`: list of two lists containing sorted (by subject number) paths to imaging files, first list corresponds to reward condition of PRL task and second list corresponds to punishment condition of PRL task
- `conf_files`: list of two lists containing sorted (by subject number) paths to confound files
- `mask_files`: brain mask files for fmri sequencnes

In [ ]:
path_bids = os.path.join(path_root, 'data/main_fmri_study')

layout = BIDSLayout(
    root=path_bids,
    derivatives=True,
    validate=True,
    index_metadata=False
)

anat_filter = {
    "extension": [".nii.gz"],
    "space": "MNI152NLin2009cAsym",
    "suffix": "T1w",
    "desc": "preproc",
    "return_type": "filename"
}

fmri_filter = {
    "extension": [".nii", ".nii.gz"],
    "space": "MNI152NLin2009cAsym",
    "suffix": "bold",
    "desc": "preproc",
    "return_type": "filename"
}

conf_filter = {
    "extension": "tsv",
    "desc": "confounds",
    "return_type": "filename"
}

mask_filter = {
    "extension": [".nii.gz"],
    "space": "MNI152NLin2009cAsym",
    "desc": "brain",
    "suffix": "mask",
    "return_type": "filename"
}

anat_files = layout.get(**anat_filter)

fmri_files, conf_files, mask_files = [], [], []

for task_dict in [{"task": "prlrew"}, {"task": "prlpun"}]:
    fmri_filter.update(task_dict)
    conf_filter.update(task_dict)
    mask_filter.update(task_dict)
    fmri_files.append(layout.get(**fmri_filter))
    conf_files.append(layout.get(**conf_filter))
    mask_files.append(layout.get(**mask_filter))

### Load task modulations
Here, modulations for model-based fMRI are downloaded. Two modulations apply to decision phase (expected probability of choosing correct box `wcor` and Pascalian expected value `exvl`) and one modulation apply to outcome phase (probabilistic prediction error `perr`).

In [ ]:
path_modulations = os.path.join(path_root, 
                                'data/main_fmri_study/derivatives/nistats/modulations')

# Load parametric modulations
modulations_wcor = np.load(os.path.join(path_modulations,'modulations_wcor.npy'))
modulations_exvl = np.load(os.path.join(path_modulations,'modulations_exvl.npy'))
modulations_perr = np.load(os.path.join(path_modulations,'modulations_perr.npy'))

### Single subject analysis

Here, first level GLM analysis is performed for each subject and task condition. `FirstLevelModel` instance is created initialized with proper GLM settings (hemodynamic response function, drift and noise model, high pass filter, smoothing kernel). Then, for each imaging sequence following steps are applied:
1. Data files are loaded (T1w anatomical image, EPI sequence, brain mask for EPI sequence, confounds table.
2. Task events onsets are loaded.
3. Different task regressors are created. Some of them are parametrically modulated using subject-specific modulations.
4. First level design matrix is created. Design matrix consists of task regressors, drift regressors, confound regressors.
5. GLM parameters are estimated.
6. Statistical maps for specified contrasts of interest are calculated and saved within `derivatives/nistats/first_level_output` directory.

In [ ]:
# Specify GLM
fmri_glm = FirstLevelModel(
    t_r=2,
    hrf_model='spm',
    drift_model='cosine',
    noise_model='ar1',
    high_pass=0.0078125, 
    standardize=True,
    smoothing_fwhm=6)

# Name of relevant confounds
confounds_relevant = [col for col in pd.read_csv(conf_files[0][0], sep='\t').columns 
                      if 'rot' in col or 'trans' in col]

# Times of image acquisition in seconds
n_scans, t_r = 730, 2
frame_times = np.arange(n_scans) * t_r

# Useful object for representing contrasts
MyContrast = namedtuple('Contrast', 'name contrast')

### Model specification

#### Task regressors

|   phase  | name               |    variable    |    onset    | duration |    modulation   |
|:--------:|--------------------|:--------------:|:-----------:|:--------:|:---------------:|
| decision | decision onset     |  `reg_dec_ons` | `onset_dec` |     0    |       None      |
| response | left button press  |  `reg_res_lbp` | `onset_res` |     0    |       None      |
|          | right button press |  `reg_res_rbp` | `onset_res` |     0    |       None      |
|          | too late feedback  | `reg_res_miss` | `onset_isi` |     0    |       None      |
|  outcome | outcome onset      |  `reg_out_ons` | `onset_out` |     0    |       None      |
|          | prediction error   | `reg_out_perr` | `onset_out` |     0    | `perr_demeaned` |

#### Confounds

As confounds, we included 24 motion parameters (6 rigid body parameters, their second power, their derivatives, second power of their derivatives).

#### Drift model

We included standard `nistats` cosine model.

In [ ]:
for sub_idx in range(n_subjects):
    for con_idx in range(n_conditions):

        # Load subject data
        anat_img = nib.load(anat_files[sub_idx])
        fmri_img = nib.load(fmri_files[con_idx][sub_idx])
        fmri_glm.mask = nib.load(mask_files[0][0])
        fmri_glm.subject_label = meta['dim1'][sub_idx]
        confounds = pd.read_csv(conf_files[con_idx][sub_idx], sep='\t')
        confounds = confounds[confounds_relevant]
        confounds.index = frame_times # Standard time representation (in seconds)

        # Setup events
        resp_type = beh[sub_idx, con_idx, :, meta['dim4'].index('response')]
        onset_dec = beh[sub_idx, con_idx, :, meta['dim4'].index('onset_dec')] 
        onset_res = beh[sub_idx, con_idx, :, meta['dim4'].index('onset_dec')] + \
                    beh[sub_idx, con_idx, :, meta['dim4'].index('rt')]
        onset_out = beh[sub_idx, con_idx, :, meta['dim4'].index('onset_out')]

        modulation_wcor = modulations_perr[sub_idx, con_idx, resp_type != 0]
        modulation_exvl = modulations_perr[sub_idx, con_idx, resp_type != 0]
        modulation_perr = modulations_perr[sub_idx, con_idx, resp_type != 0]

        modulation_wcor_demeaned = modulation_wcor - np.mean(modulation_wcor)
        modulation_exvl_demeaned = modulation_exvl - np.mean(modulation_exvl)
        modulation_perr_demeaned = modulation_perr - np.mean(modulation_perr)

        # Create regressors
        reg_res_lbp = Regressor('res_lbp', frame_times, onset_res[resp_type==-1])
        reg_res_rbp = Regressor('res_rbp', frame_times, onset_res[resp_type==1])
        reg_res_miss = Regressor('res_miss', frame_times, onset_dec[resp_type==0] + 2)

        reg_dec_ons = Regressor('dec_ons', frame_times, onset_dec[resp_type != 0])
        reg_dec_tillres = Regressor(
            'dec_tillres', frame_times, onset_dec[resp_type != 0],
            duration=beh[sub_idx, con_idx, resp_type != 0, meta['dim4'].index('rt')])
        reg_dec_wcor = Regressor(
            'dec_wcor', frame_times, onset_dec[resp_type != 0],
            duration=beh[sub_idx, con_idx, resp_type != 0, meta['dim4'].index('rt')],
            modulation = modulation_wcor_demeaned)

        reg_out_ons = Regressor('out_ons', frame_times, onset_out[resp_type != 0])
        reg_out_miss = Regressor('out_miss', frame_times, onset_out[resp_type == 0]) 
        reg_out_perr = Regressor('out_perr', frame_times, onset_out[resp_type != 0], 
            modulation = modulation_perr_demeaned)

        regressors = [reg_dec_ons, 
                      reg_res_lbp, reg_res_rbp, reg_res_miss, 
                      reg_out_ons, reg_out_perr]

        # Create design matrix
        dm, conditions = my_make_first_level_design_matrix(regressors)

        # Fit GLM
        fmri_glm = fmri_glm.fit(fmri_img, confounds=confounds, design_matrices=dm)

        # Define contrasts
        contrasts = []
        contrasts.append(MyContrast('res_lbp_minus_rbp', 
                                    conditions['res_lbp'] - conditions['res_rbp']))
        contrasts.append(MyContrast('out_ons', conditions['out_ons']))
        contrasts.append(MyContrast('out_perr', conditions['out_perr']))

        # Compute statistical maps and save it
        for contrast in contrasts:
            stat_map = fmri_glm.compute_contrast(
                contrast.contrast, 
                stat_type='t',
                output_type='z_score'
            )

            stat_map_fname = f"sub-{meta['dim1'][sub_idx]}_task-prl{meta['dim2'][con_idx]}_statmap"
            os.makedirs(os.path.join(path_out, contrast.name), exist_ok=True)

            nib.save(stat_map, os.path.join(path_out, contrast.name, stat_map_fname))